# 五、 合并
1. **append和assign**  
**append方法**：  
利用序列添加行  
用DataFrame添加表    
**asssign方法**：
添加列  
添加多列  
   
   
2. **combine与update**  
combine和update都是用于表的填充函数，可以根据某种规则填充。  
**combine方法：**  
*填充对象*：  
按表的顺序轮流进行逐列循环，并且自动索引对齐   
*combine_first方法*：  
作用是用df2填补df1的缺失值，功能比较简单，但很多时候会比combine更常用  
**update方法：**  
*三个特性*：  
返回的框索引只会与被调用框的一致    
第二个框中的nan元素不会起作用  
没有返回值，直接在df上操作  
  
  
3. **concat方法**  
concat方法可以在两个维度上拼接，默认纵向凭借（axis=0），拼接方式默认外连接  
所谓外连接，就是取拼接方向的并集，而'inner'时取拼接方向（若使用默认的纵向拼接，则为列的交集）的交集   
   
   
4. **merge与join**  
**merge函数：**    
作用是将两个pandas对象横向合并，遇到重复的索引项时会使用笛卡尔积，默认inner连接，可选left、outer、right连接  
所谓左连接，就是指以第一个表索引为基准，右边的表中如果不再左边的则不加入，如果在左边的就以笛卡尔积的方式加入  
merge/join与concat的不同之处在于on参数，可以指定某一个对象为key来进行连接  
**join函数：**  
join函数作用是将多个pandas对象横向拼接，遇到重复的索引项时会使用笛卡尔积，默认左连接，可选inner、outer、right连接

In [3]:
import numpy as np
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

##  问题与练习
### 1. 问题

#### 【问题一】 
请思考什么是append/assign/combine/update/concat/merge/join各自最适合使用的场景，并举出相应的例子。
#### 【问题二】 
merge_ordered和merge_asof的作用是什么？和merge是什么关系？
#### 【问题三】 
请构造一个多级索引与多级索引合并的例子，尝试使用不同的合并函数。
#### 【问题四】 
上文提到了连接的笛卡尔积，那么当连接方式变化时（inner/outer/left/right），这种笛卡尔积规则会相应变化吗？请构造相应例子。
    
 

### 2. 练习

#### 【练习一】 
有2张公司的员工信息表，每个公司共有16名员工，共有五个公司，请解决如下问题：


In [7]:
pd.read_csv('./data/Employee1.csv').head()
pd.read_csv('./data/Employee2.csv').head()
Employee1 = pd.read_csv('./data/Employee1.csv')
Employee2 = pd.read_csv('./data/Employee2.csv')

,Company,Name,Age,Height,Weight,Salary
0,A,a1,47,188,63.7,25819
1,A,a3,39,172,55.9,21983
2,A,a4,43,158,62.5,21755
3,A,a6,42,182,76.9,17354
4,A,a7,49,171,94.6,6177


,Company,Name,Age,Height,Weight,Salary
0,A,a1,30,156,91.2,28133
1,A,a2,50,190,83.4,6673
2,A,a3,34,168,96.6,16503
3,A,a5,51,176,97.2,23294
4,A,a6,37,183,93.2,19256


(a) 每个公司有多少员工满足如下条件：既出现第一张表，又出现在第二张表。  
(b) 将所有不符合(a)中条件的行筛选出来，合并为一张新表，列名与原表一致。  
(c) 现在需要编制所有80位员工的信息表，对于(b)中的员工要求不变，对于满足(a)条件员工，它们在某个指标的数值，取偏离它所属公司中满足(b)员工的均值数较小的哪一个，  
例如：P公司在两张表的交集为{p1}，并集扣除交集为{p2,p3,p4}，那么如果后者集合的工资均值为1万元，且p1在表1的工资为13000元，在表2的工资为9000元，那么应该最后取9000元作为p1的工资，最后对于没有信息的员工，利用缺失值填充。

In [11]:
# (a) 每个公司有多少员工满足如下条件：既出现第一张表，又出现在第二张表。
Name = []
for name in Employee1['Name']:
    if name in list(Employee2['Name']):
        Name.append(name)
Name

# 参考答案方法
# L = list(set(df1['Name']).intersection(set(df2['Name'])))


['a1',
 'a3',
 'a6',
 'b1',
 'b3',
 'b7',
 'b15',
 'c3',
 'c10',
 'c12',
 'c13',
 'd5',
 'd10',
 'e8',
 'e10',
 'e11']

In [15]:
# (b) 将所有不符合(a)中条件的行筛选出来，合并为一张新表，列名与原表一致。  
New_Employee1 = Employee1[~Employee1['Name'].isin(Name)]
New_Employee2 = Employee2[~Employee2['Name'].isin(Name)]
New_Employee = pd.concat([New_Employee1,New_Employee2]).set_index('Name')
New_Employee.head()
New_Employee.shape

,Company,Age,Height,Weight,Salary
Name,,,,,
a4,A,43,158,62.5,21755
a7,A,49,171,94.6,6177
a8,A,51,168,89.5,3246
a9,A,36,186,62.8,3569
a13,A,58,190,75.9,21854


(45, 5)

In [16]:
# (c) 现在需要编制所有80位员工的信息表，对于(b)中的员工要求不变，对于满足(a)条件员工，它们在某个指标的数值，取偏离它所属公司中满足(b)员工的均值数较小的哪一个，  





#### 【练习二】 
有2张课程的分数表（分数随机生成），但专业课（学科基础课、专业必修课、专业选修课）与其他课程混在一起，请解决如下问题：

In [22]:
Course1 = pd.read_csv('data/Course1.csv')
Course2 = pd.read_csv('data/Course2.csv')
pd.read_csv('data/Course1.csv').head()
pd.read_csv('data/Course2.csv').head()

,课程名字,课程类别,学分,分数
0,思想道德修养与法律基础,思政类,3,89.0
1,云计算应用与开发,专业选修课,3,96.0
2,社会计算,专业选修课,3,78.0
3,深度学习,专业选修课,3,75.0
4,人工智能导论,专业必修课,3,84.0


,课程名字,课程类别,学分,分数
0,高等数学（一）,学科基础课,4,99.0
1,数据科学与工程导论,学科基础课,3,NaN
2,专业英语,学科基础课,2,100.0
3,概率论,学科基础课,3,99.0
4,计算机系统,专业必修课,4,80.0


 (a) 将两张表分别拆分为专业课与非专业课（结果为四张表）。  
 (b) 将两张专业课的分数表和两张非专业课的分数表分别合并。  
 (c) 不使用(a)中的步骤，请直接读取两张表合并后拆分。  
 (d) 专业课程中有缺失值吗，如果有的话请在完成(3)的同时，用组内（3种类型的专业课）均值填充缺失值后拆分。  

In [30]:
# (a) 将两张表分别拆分为专业课与非专业课（结果为四张表）。  
## 查看有哪些课程类别
Course1.课程类别.unique()
Course2.课程类别.unique()

Category = ["学科基础课","专业必修课","专业选修课"]
Course11 = Course1[Course1["课程类别"].isin(Category)]
Course12 = Course1[~Course1["课程类别"].isin(Category)]
Course21 = Course2[Course2["课程类别"].isin(Category)]
Course22 = Course2[~Course2["课程类别"].isin(Category)]
Course11.head()
Course12.head()
Course21.head()
Course22.head()

array(['思政类', '专业选修课', '专业必修课', '学科基础课', '体育类', '公共任意选修类', '英语类', '精品课程'],
      dtype=object)

array(['学科基础课', '专业必修课', '专业选修课', '英语类', '思政类', '文化传承类', '公共任意选修类'],
      dtype=object)

,课程名字,课程类别,学分,分数
1,云计算应用与开发,专业选修课,3,96.0
2,社会计算,专业选修课,3,78.0
3,深度学习,专业选修课,3,75.0
4,人工智能导论,专业必修课,3,84.0
6,数据结构与算法,学科基础课,5,82.0


,课程名字,课程类别,学分,分数
0,思想道德修养与法律基础,思政类,3,89.0
5,中国近代史纲要,思政类,3,97.0
8,网球（初）,体育类,1,81.0
10,极端性气候与陆地生态系统,公共任意选修类,2,78.0
13,游泳（初）,体育类,1,75.0


,课程名字,课程类别,学分,分数
0,高等数学（一）,学科基础课,4,99.0
1,数据科学与工程导论,学科基础课,3,NaN
2,专业英语,学科基础课,2,100.0
3,概率论,学科基础课,3,99.0
4,计算机系统,专业必修课,4,80.0


,课程名字,课程类别,学分,分数
25,学术英语听说（二）,英语类,2,92.0
26,学术英语阅读,英语类,2,72.0
27,学术英语写作,英语类,2,98.0
28,美国社会与文化,英语类,2,77.0
29,马克思主义基本原理概论,思政类,3,95.0


In [33]:
# (b) 将两张专业课的分数表和两张非专业课的分数表分别合并。  
Course_pro = pd.concat([Course11,Course21])
Course_common = pd.concat([Course12,Course22])
Course_pro.head()
Course_common.head()

,课程名字,课程类别,学分,分数
1,云计算应用与开发,专业选修课,3,96.0
2,社会计算,专业选修课,3,78.0
3,深度学习,专业选修课,3,75.0
4,人工智能导论,专业必修课,3,84.0
6,数据结构与算法,学科基础课,5,82.0


,课程名字,课程类别,学分,分数
0,思想道德修养与法律基础,思政类,3,89.0
5,中国近代史纲要,思政类,3,97.0
8,网球（初）,体育类,1,81.0
10,极端性气候与陆地生态系统,公共任意选修类,2,78.0
13,游泳（初）,体育类,1,75.0


In [34]:
# (c) 不使用(a)中的步骤，请直接读取两张表合并后拆分。  
Course = pd.concat([Course1,Course2])
Course_pro= Course.query('课程类别 in ["学科基础课","专业必修课","专业选修课"]')
Course_common= Course.query('课程类别 not in ["学科基础课","专业必修课","专业选修课"]')
Course_pro.head()
Course_common.head()

,课程名字,课程类别,学分,分数
1,云计算应用与开发,专业选修课,3,96.0
2,社会计算,专业选修课,3,78.0
3,深度学习,专业选修课,3,75.0
4,人工智能导论,专业必修课,3,84.0
6,数据结构与算法,学科基础课,5,82.0


,课程名字,课程类别,学分,分数
0,思想道德修养与法律基础,思政类,3,89.0
5,中国近代史纲要,思政类,3,97.0
8,网球（初）,体育类,1,81.0
10,极端性气候与陆地生态系统,公共任意选修类,2,78.0
13,游泳（初）,体育类,1,75.0


In [39]:
# (d) 专业课程中有缺失值吗，如果有的话请在完成(3)的同时，用组内（3种类型的专业课）均值填充缺失值后拆分。 
Course[Course.分数.isnull()]

Course_pro['分数'] = Course_pro.groupby('课程类别').transform(lambda x: x.fillna(x.mean()))['分数']
Course_pro= Course.query('课程类别 in ["学科基础课","专业必修课","专业选修课"]')
Course_common= Course.query('课程类别 not in ["学科基础课","专业必修课","专业选修课"]')
Course_pro.head()
Course_common.head()


,课程名字,课程类别,学分,分数
14,计算机视觉,专业选修课,3,NaN
1,数据科学与工程导论,学科基础课,3,NaN
11,数据伦理,专业必修课,1,NaN


C:\Users\never\Miniconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,课程名字,课程类别,学分,分数
1,云计算应用与开发,专业选修课,3,96.0
2,社会计算,专业选修课,3,78.0
3,深度学习,专业选修课,3,75.0
4,人工智能导论,专业必修课,3,84.0
6,数据结构与算法,学科基础课,5,82.0


,课程名字,课程类别,学分,分数
0,思想道德修养与法律基础,思政类,3,89.0
5,中国近代史纲要,思政类,3,97.0
8,网球（初）,体育类,1,81.0
10,极端性气候与陆地生态系统,公共任意选修类,2,78.0
13,游泳（初）,体育类,1,75.0
